In [ ]:
from elasticsearch import Elasticsearch
import json
from kafka import KafkaProducer
from json import dumps
import time

'''Purpose: Originally, records were retrieved from Twitter and Reddit and stored directly in Elasticsearch without assigning 
sentiment scores. This code extracts all records from original index and sends them into the Kafka-Spark-Elasticsearch pipeline
so that they are properly assigned a sentiment score and reindexed. 
'''
def main():    
    producer = KafkaProducer(bootstrap_servers=['35.232.117.118:9092'], 
                            value_serializer=lambda x: dumps(x).encode('utf-8'),
                            buffer_memory = 200000000, 
                            linger_ms = 500,
                            api_version = (0,10))

    es = Elasticsearch('34.73.60.209:9200')

    es_query = {
            'size' : 10000,
            'query': {
                'match_all' : {}
           }
       }

    result = es.search(
        index='trending', 
        doc_type='_doc', 
        body =es_query,
        scroll="1m")

    scrollId = result['_scroll_id']

    scrollSize = result['hits']['total']
    print('Total records: ', scrollSize)
    docs_sent = 0
    while scrollSize > 0 :
        for i in range (0, len(result['hits']['hits'])) :
            data = result['hits']['hits'][i]
            data['_source']['id'] = data['_id']
            data = data['_source']
            producer.send('trending', value=data)
            scrollSize -= 1
        
        time.sleep(5)
        docs_sent+= 10000
        print('Remaining Documents: ',scrollSize)
        print('Documents Sent: ',docs_sent)
        result = es.scroll(scroll_id=scrollId, scroll='1m')
        
if __name__ == '__main__':
   main()